In [77]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from random import randint
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.seq2seq import (
    Seq2SeqModel,
    Seq2SeqArgs,
)

In [3]:
dataset = load_dataset("D:\\CSN")

In [86]:
model_args = ClassificationArgs(
    sliding_window=True,
    labels_list=[0, 1],
    learning_rate=1e-5,
    train_batch_size=64,
    max_seq_length=200,
    num_train_epochs=8,
    optimizer="Adam",
)
model = ClassificationModel(
    "roberta", "microsoft/codebert-base", args=model_args, use_cuda=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
train_data = []
test_data = []

shuffled_data = dataset["train"].shuffle(seed=42)

for index in range(0, 3000, 3):
    true = randint(1, 10) > 5
    if true:
        row = [
            shuffled_data._getitem(index + 1)["func_documentation_string"]
            + "\n"
            + shuffled_data._getitem(index)["func_code_string"],
            1,
        ]
    else:
        row = [
            shuffled_data._getitem(index)["func_documentation_string"]
            + "\n"
            + shuffled_data._getitem(index)["func_code_string"],
            0,
        ]

    train_data.append(row)

for index in range(3001, 6000, 3):
    true = randint(1, 10) > 5
    if true:
        row = [
            shuffled_data._getitem(index + 1)["func_documentation_string"]
            + "\n"
            + shuffled_data._getitem(index)["func_code_string"],
            1,
        ]
    else:
        row = [
            shuffled_data._getitem(index)["func_documentation_string"]
            + "\n"
            + shuffled_data._getitem(index)["func_code_string"],
            0,
        ]

    test_data.append(row)

train_data = pd.DataFrame(train_data, columns=["text", "labels"])
test_data = pd.DataFrame(test_data, columns=["text", "labels"])
display(test_data.head(5))

,text,labels
0,"Set the turn column corresponding to 3,4 or 5 ...",1
1,Use this to clear the variables in a section o...,0
2,returns union of this range with given range.<...,1
3,Creates a new instance of the setup Bean.<p>\n...,0
4,"$ANTLR start ""EXPONENT""\npublic void marshall(...",1


In [59]:
train_data.to_csv("test.csv", index=False)

In [61]:
display(train_data.head(5))

,text,labels
0,Returns the message for a given upload variant...,false
1,If there is a parent class loader use the supe...,true
2,Returns the last commerce notification queue e...,false
3,Moves to the next record in the current scan. ...,true
4,Shift a property.\n\n@param _key key the prope...,false


In [73]:
train_data.iloc[:, 1].astype(str)

0       true
1      false
2      false
3      false
4      false
       ...  
995    false
996     true
997     true
998    false
999     true
Name: labels, Length: 1000, dtype: object

In [75]:
model.train_model(train_data, output_dir="./output", eval_df=test_data)

model.eval_model(test_data)

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/531 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

({'mcc': 0.25848072690894125,
  'tp': 393,
  'tn': 230,
  'fp': 268,
  'fn': 109,
  'eval_loss': 0.7189379736434581},
 [array([[-0.98632812,  1.1015625 ],
         [ 0.02062988,  0.17858887],
         [-0.21508789,  0.36962891]]),
  array([[ 0.96582031, -0.89648438],
         [-0.34667969,  0.54199219]]),
  array([[ 0.03707886,  0.13793945],
         [ 0.953125  , -0.93603516],
         [ 1.27050781, -1.18847656],
         [ 0.22290039, -0.1817627 ],
         [ 1.45410156, -1.47363281],
         [ 1.84570312, -1.8984375 ],
         [ 1.0625    , -0.86035156],
         [ 0.40673828, -0.10668945],
         [ 0.16223145,  0.03240967],
         [ 0.44360352, -0.29638672],
         [ 0.80517578, -0.703125  ],
         [ 0.78564453, -0.53417969],
         [ 1.328125  , -1.13671875],
         [ 1.0859375 , -0.93652344],
         [ 0.61181641, -0.53466797],
         [ 0.51318359, -0.37768555],
         [ 0.20996094, -0.14099121],
         [ 0.95117188, -0.76855469],
         [ 0.78808594, -0.6